In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings("ignore")

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [10]:
sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e7/sample_submission.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")
train = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")

In [11]:
train['Stage_fear'] = train['Stage_fear'].replace({'Yes': 1, 'No': 0})
train['Drained_after_socializing'] = train['Drained_after_socializing'].replace({'Yes': 1, 'No': 0})
test['Stage_fear'] = test['Stage_fear'].replace({'Yes': 1, 'No': 0})
test['Drained_after_socializing'] = test['Drained_after_socializing'].replace({'Yes': 1, 'No': 0})
train['Personality'] = train['Personality'].replace({'Extrovert': 1, 'Introvert': 0})
train

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,0.0,6.0,4.0,0.0,15.0,5.0,1
1,1,1.0,0.0,7.0,3.0,0.0,10.0,8.0,1
2,2,6.0,1.0,1.0,0.0,NaN,3.0,0.0,0
3,3,3.0,0.0,7.0,3.0,0.0,11.0,5.0,1
4,4,1.0,0.0,4.0,4.0,0.0,13.0,NaN,1
...,...,...,...,...,...,...,...,...,...
18519,18519,3.0,0.0,7.0,3.0,0.0,9.0,7.0,1
18520,18520,1.0,NaN,6.0,7.0,0.0,6.0,5.0,1
18521,18521,7.0,1.0,1.0,1.0,1.0,1.0,NaN,0
18522,18522,NaN,1.0,1.0,0.0,1.0,5.0,2.0,0


In [18]:
numeric_cols = train.select_dtypes(include=[np.number]).columns
missing_before = train[numeric_cols].isnull().sum().sum()
missing_before

9196

In [19]:
from sklearn.impute import KNNImputer
imputer = KNNImputer()
imputed_data = imputer.fit_transform(train[numeric_cols])

In [28]:
imputed_df = pd.DataFrame(imputed_data, columns=numeric_cols, index = train.index)
imputed_df[['Stage_fear','Drained_after_socializing']] = imputed_df[['Stage_fear','Drained_after_socializing']].round()
imputed_df[(imputed_df['Stage_fear'] == imputed_df['Personality'])]

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
63,63.0,8.0,1.0,4.0,2.0,1.0,4.2,0.0,1.0
102,102.0,6.0,1.0,2.0,0.0,1.0,2.0,0.0,1.0
124,124.0,4.0,0.0,7.0,6.0,0.0,11.0,3.0,0.0
139,139.0,11.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0
149,149.0,0.0,0.0,4.0,6.0,0.0,8.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...
18377,18377.0,5.0,1.0,2.0,0.0,1.0,7.0,1.0,1.0
18437,18437.0,11.0,1.0,0.0,2.0,1.0,0.0,3.0,1.0
18443,18443.0,1.8,0.0,7.0,4.0,0.0,9.0,4.0,0.0
18485,18485.0,8.0,0.0,0.0,2.0,1.0,3.0,2.0,0.0


In [ ]:
(imputed_df[['Stage_fear','Drained_after_socializing']] == 0) | (imputed_df[['Stage_fear','Drained_after_socializing']] == 1)

In [16]:
(train['Personality'] == 1).mean()*100

73.95270999784064

In [4]:
train.isna().mean(axis = 0)*100 #% what percent of the data is NaN , in each column

id                            0.000000
Time_spent_Alone              6.424098
Stage_fear                   10.219175
Social_event_attendance       6.370114
Going_outside                 7.914057
Drained_after_socializing     6.202764
Friends_circle_size           5.689916
Post_frequency                6.823580
Personality                   0.000000
dtype: float64

In [5]:
df = train[train.isna().any(axis=1) == 0]
df.describe()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
count,10189.000000,10189.000000,10189.000000,10189.000000,10189.000000,10189.000000,10189.000000,10189.000000,10189.000000
mean,9260.691530,2.633134,0.166945,5.681519,4.444793,0.167043,8.503092,5.500834,0.826578
std,5345.623063,2.630621,0.372945,2.597142,1.884790,0.373032,4.073302,2.709847,0.378631
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4653.000000,1.000000,0.000000,4.000000,3.000000,0.000000,5.000000,3.000000,1.000000
50%,9213.000000,2.000000,0.000000,6.000000,5.000000,0.000000,9.000000,6.000000,1.000000
75%,13878.000000,3.000000,0.000000,8.000000,6.000000,0.000000,12.000000,8.000000,1.000000
max,18523.000000,11.000000,1.000000,10.000000,7.000000,1.000000,15.000000,10.000000,1.000000


In [6]:
df = df.drop(df[(df['Stage_fear'] == df['Personality']) & (df['Drained_after_socializing'] == df['Personality'])].index)
df

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,0.0,6.0,4.0,0.0,15.0,5.0,1
1,1,1.0,0.0,7.0,3.0,0.0,10.0,8.0,1
3,3,3.0,0.0,7.0,3.0,0.0,11.0,5.0,1
7,7,2.0,0.0,8.0,3.0,0.0,4.0,5.0,1
9,9,1.0,0.0,8.0,6.0,0.0,14.0,9.0,1
...,...,...,...,...,...,...,...,...,...
18509,18509,1.0,0.0,3.0,4.0,0.0,15.0,4.0,1
18511,18511,0.0,0.0,4.0,5.0,0.0,11.0,4.0,1
18514,18514,6.0,0.0,5.0,3.0,0.0,10.0,4.0,1
18519,18519,3.0,0.0,7.0,3.0,0.0,9.0,7.0,1


In [ ]:
df['Personality'].mean()*100

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=) 